## Steps
1. Upload a self-storage file
2. Parse out a set of the lat/lon
3. Download a dataset of the images for all lat/lons
4. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import io as b_io
import os
from pathlib import Path

import dill

# Import necessary packages
from mosaiks import config as c
from mosaiks import transforms
from mosaiks.utils.imports import *

env variable MOSAIKS_HOME not defined; setting to: "/home/ubuntu/cs230/mosaiks-paper"
If not desired, please reset os.environ["MOSAIKS_NAME"]


In [203]:
import requests
import pandas as pd
from tqdm import tqdm
import torch
from torchvision import models
from torchvision import transforms
import dill
import time
import skimage.io
import skimage.transform
from mosaiks.featurization import chunks
import io as python_io

In [24]:
filename = "train_df.tsv"
filename = c.data_dir + "/raw/" + filename

data = pd.read_csv(filename, sep='\t', header=0, index_col=False)
data

,price,domain,website,places_id,address,city,state,country,dimensions,dim1,...,oversized_doors,power_outlet,premium_location,rv_parking,parking,rv_parking_covered,rv_parking_uncovered,stair_access,unusual_size,wine_storage
0,72.0,storagesense.com,https://www.storagesense.com/location/usa/tx/h...,ChIJkxtdA8mzQIYROlwVeGwwh6I,18006 US-59,Humble,TX,United States,5' x 5',5.0,...,0,0,0,0,0,0,0,1,0,0
1,77.0,storagesense.com,https://www.storagesense.com/location/usa/tx/h...,ChIJkxtdA8mzQIYROlwVeGwwh6I,18006 US-59,Humble,TX,United States,5' x 5',5.0,...,0,0,0,0,0,0,0,0,0,0
2,105.0,storagesense.com,https://www.storagesense.com/location/usa/tx/h...,ChIJkxtdA8mzQIYROlwVeGwwh6I,18006 US-59,Humble,TX,United States,5' x 10',5.0,...,0,0,0,0,0,0,0,1,0,0
3,119.0,storagesense.com,https://www.storagesense.com/location/usa/tx/h...,ChIJkxtdA8mzQIYROlwVeGwwh6I,18006 US-59,Humble,TX,United States,5' x 10',5.0,...,0,0,0,0,0,0,0,0,0,0
4,118.0,storagesense.com,https://www.storagesense.com/location/usa/tx/h...,ChIJkxtdA8mzQIYROlwVeGwwh6I,18006 US-59,Humble,TX,United States,5' x 15',5.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22096,19.0,extraspace.com,https://www.extraspace.com/storage/facilities/...,ChIJEdlQ-KvV2IcRhNRyDF9stDc,1289 Vero Ln,Ellisville,MO,United States,5' x 5',5.0,...,0,0,0,0,0,0,0,0,1,0
22097,37.0,extraspace.com,https://www.extraspace.com/storage/facilities/...,ChIJEdlQ-KvV2IcRhNRyDF9stDc,1289 Vero Ln,Ellisville,MO,United States,4' x 5',4.0,...,0,0,0,0,0,0,0,0,0,0
22098,35.0,extraspace.com,https://www.extraspace.com/storage/facilities/...,ChIJEdlQ-KvV2IcRhNRyDF9stDc,1289 Vero Ln,Ellisville,MO,United States,5' x 5',5.0,...,0,0,0,0,0,0,0,0,0,0
22099,193.0,extraspace.com,https://www.extraspace.com/storage/facilities/...,ChIJEdlQ-KvV2IcRhNRyDF9stDc,1289 Vero Ln,Ellisville,MO,United States,10' x 11',10.0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
latlons = data[['latitude', 'longitude', 'places_id']].drop_duplicates()
latlons = latlons.to_numpy()
print(latlons)

[[29.9796496 -95.2753524 'ChIJkxtdA8mzQIYROlwVeGwwh6I']
 [42.6765096 -84.5239802 'ChIJBXG-W2DBIogRs552_nWUOws']
 [30.6371446 -96.3810666 'ChIJJ6ZCoiSCRoYRjFNyzETgjMc']
 ...
 [43.6897019 -116.6167439 'ChIJYUFELt6zr1QRj4ciTNbBr1c']
 [38.3391359 -76.561331 'ChIJPeCXddVlt4kRaAvPcjj2HwU']
 [38.5971049 -90.585272 'ChIJEdlQ-KvV2IcRhNRyDF9stDc']]


In [72]:
api_key = "<insert api key here>"
url = "https://maps.googleapis.com/maps/api/staticmap?"
output_path = "../../data/raw/imagery/self-storage/"

In [73]:
# Helper function to generate the end file name 
def generate_image_file_name(lat, lon, zoom, pix):
  return f"{lat}_{lon}_{zoom}_{pix}_{pix}.png"

# test function to verify file name output
def print_output_file_names(latlons, limit):
  for latlon in latlons[:limit]:
    lat, lon = latlon[0], latlon[1]
    print(generate_image_file_name(lat, lon, 16, 640))
    
# function to actually download the images
def download_images(latlons, start=0, end=0):
  # get and save the images
  for idx in tqdm(range(start, end)):
    latlon = latlons[idx]
    lat, lon = latlon[0], latlon[1]
    request_url = url + f"center={lat},{lon}&zoom={16}&size={640}x{640}&maptype=satellite&key={api_key}"
    r = requests.get(url + f"center={lat},{lon}&zoom={16}&size={640}x{640}&maptype=satellite&key={api_key}")
    if (r.status_code != 200):
        print(f"Could not download idx {idx}, http code {r.status_code}")
        break
    filename = output_path + generate_image_file_name(lat, lon, 16, 640)
    with open(filename, 'wb') as f:
      f.write(r.content)

In [74]:
print_output_file_names(latlons, 10)
print(latlons.shape)
# download_images(latlons, 0, latlons.shape[0])

29.9796496_-95.2753524_16_640_640.png
42.6765096_-84.5239802_16_640_640.png
30.6371446_-96.3810666_16_640_640.png
35.9843818_-78.5389999_16_640_640.png
39.9787487_-76.7331381_16_640_640.png
34.7439492_-86.5976824_16_640_640.png
33.550876_-84.3705764_16_640_640.png
42.455787_-82.962877_16_640_640.png
35.767771_-80.8561524_16_640_640.png
36.0670826_-79.4254095_16_640_640.png
(1707, 3)


In [75]:
filename = "test_id_df.tsv"
filename = c.data_dir + "/raw/" + filename

data_test_id = pd.read_csv(filename, sep='\t', header=0, index_col=False)

latlons_test_id = data_test_id[['latitude', 'longitude', 'places_id']].drop_duplicates()
latlons_test_id = latlons_test_id.to_numpy()
print(latlons_test_id)

[[42.9750012 -73.8608553 'ChIJHz19G2ZA3okR7MeMy1ZyHqg']
 [30.3587044 -89.1606283 'ChIJ6wcat7w8nIgR055G7AZlRUs']
 [32.7674327 -97.4071377 'ChIJG87HxGJzToYRj1wV2KeAF2w']
 ...
 [33.6976813 -117.8172083 'ChIJT5IbDwPd3IAR06KtP9VK1HI']
 [38.456701 -122.6774424 'ChIJm2syBz1HhIARPITVa6_pkXs']
 [42.1509105 -72.4577793 'ChIJi_oB-UPD5okR_C1mlcYO-hk']]


In [76]:
print_output_file_names(latlons_test_id, 10)
print(latlons_test_id.shape)

42.9750012_-73.8608553_16_640_640.png
30.3587044_-89.1606283_16_640_640.png
32.7674327_-97.4071377_16_640_640.png
42.7469932_-84.5420907_16_640_640.png
30.0518011_-95.4375312_16_640_640.png
41.6381235_-88.0788409_16_640_640.png
40.3762789_-105.0789257_16_640_640.png
33.4603839_-82.183918_16_640_640.png
40.2345049_-75.6230819_16_640_640.png
42.3798917_-83.3148246_16_640_640.png
(427, 3)


In [77]:
# download_images(latlons_test_id, 0, latlons_test_id.shape[0])

100%|███████████████████████████████████████████████████████████████████████████████| 427/427 [01:58<00:00,  3.62it/s]


In [79]:
filename = "test_ood_df.tsv"
filename = c.data_dir + "/raw/" + filename

data_test_ood = pd.read_csv(filename, sep='\t', header=0, index_col=False)

latlons_test_ood = data_test_ood[['latitude', 'longitude', 'places_id']].drop_duplicates()
latlons_test_ood = latlons_test_ood.to_numpy()
print_output_file_names(latlons_test_ood, 10)
print(latlons_test_ood.shape)

40.8229805_-72.9790197_16_640_640.png
43.124286_-73.768612_16_640_640.png
43.1653212_-73.7051917_16_640_640.png
43.1239665_-70.7648303_16_640_640.png
36.7880659_-76.0656629_16_640_640.png
36.8018954_-76.3811516_16_640_640.png
43.3165168_-73.5950938_16_640_640.png
42.4973221_-70.9246327_16_640_640.png
38.0174119_-78.4176878_16_640_640.png
41.346803_-73.2568649_16_640_640.png
(162, 3)


In [80]:
# download_images(latlons_test_ood, 0, latlons_test_ood.shape[0])

100%|███████████████████████████████████████████████████████████████████████████████| 162/162 [00:46<00:00,  3.48it/s]


## Now we featurize
Let's just use the pretrained resnet152 for now, since the self storage data doesn't fit nicely into MOSAIKS

In [183]:
class SelfStorageImageDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        data_home,
        latlons,
    ):
        self.latlons = latlons
        self.data_home = data_home
#         self.transform=transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.latlons)

    def __getitem__(self, i):
        latlon = self.latlons[i]
        im = io.load_img_from_local(latlon, self.data_home, 16, 640)
        if len(im.shape) < 3:
            im = np.stack((im, im, im), axis=2)
        if im.shape[-1] > 3:
            im = im[:, :, :3]
        if im.shape[-1] == 1:
            im = np.concatenate((im, im, im), axis=2)
        x_i = im.transpose(2, 0, 1)
        return x_i

In [184]:
print(latlons.shape)
print(latlons_test_id.shape)
print(latlons_test_ood.shape)

# We create a combined dataset
all_latlons = np.concatenate((latlons, latlons_test_id, latlons_test_ood), axis=0)
print(all_latlons.shape)
print(all_latlons[:5])

image_dir = c.data_dir + "/raw/imagery/self-storage/"

places_ids = all_latlons[:,2]
print(places_ids.shape)
print(places_ids[:5])

ll_formatted = all_latlons[:,:2]
print(ll_formatted.shape)
print(ll_formatted[:5])

(1707, 3)
(427, 3)
(162, 3)
(2296, 3)
[[29.9796496 -95.2753524 'ChIJkxtdA8mzQIYROlwVeGwwh6I']
 [42.6765096 -84.5239802 'ChIJBXG-W2DBIogRs552_nWUOws']
 [30.6371446 -96.3810666 'ChIJJ6ZCoiSCRoYRjFNyzETgjMc']
 [35.9843818 -78.5389999 'ChIJGxPXPm9TrIkR8DTsN0sjT24']
 [39.9787487 -76.7331381 'ChIJQe7mgp2OyIkRyP-yL8dVN00']]
(2296,)
['ChIJkxtdA8mzQIYROlwVeGwwh6I' 'ChIJBXG-W2DBIogRs552_nWUOws'
 'ChIJJ6ZCoiSCRoYRjFNyzETgjMc' 'ChIJGxPXPm9TrIkR8DTsN0sjT24'
 'ChIJQe7mgp2OyIkRyP-yL8dVN00']
(2296, 2)
[[29.9796496 -95.2753524]
 [42.6765096 -84.5239802]
 [30.6371446 -96.3810666]
 [35.9843818 -78.5389999]
 [39.9787487 -76.7331381]]


In [197]:
def resize_images(images):
    images_resized = []
    for im in images:
        im = im[:, :, :3]
        images_resized.append(
            skimage.transform.resize(
                im, (224, 224), mode="constant", anti_aliasing=True
            )
        )
    images = np.stack(images_resized, axis=0)
    return images

def resnet_features(images, model, batch_size=60, gpu=True):
    results = []
    if gpu:
        model = model.cuda()
    for images_chunk in chunks(images, batch_size):
        if len(images_chunk.shape) < 4:
            images_chunk = images[np.newaxis, :, :, :]
        images_chunk = images_chunk.astype("float32").transpose(0, 3, 1, 2)
        images_torch = torch.from_numpy(images_chunk)
        if gpu:
            images_torch = images_torch.cuda()
        x = model.conv1(images_torch)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)
        x = model.layer1(x)
        x = model.layer2(x)
        x = model.layer3(x)
        x = model.layer4(x)
        x = model.avgpool(x)
        x = x.view(x.size(0), -1)
        x = x.cpu().data.numpy()
        results.append(x)
    torch.cuda.empty_cache()
    return np.concatenate(results, axis=0)

def full_featurize(dataset, model_ft, batch_size):
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=1
    )
    output_features = []
    data = {}
    ids = []
    print(len(dataloader))
    for j, X_batch in enumerate(dataloader):
        print("batch:", j)
        t = time.time()
        print(X_batch.shape)
        X_batch = resize_images(X_batch.numpy())
        print(X_batch.shape)
        X_features = resnet_features(X_batch, model_ft)
        e = time.time()
        print(f"batch: {j} took {e - t}")
        output_features.append(X_features)
    bio_features = python_io.BytesIO()
    np.save(bio_features, np.vstack(output_features))
    data["X"] = bio_features.getvalue()
    return data

In [204]:
out_file = c.features_dir + "/resnet152_self_storage.pkl"
this_c = c.features["pretrained"]

model_ft = getattr(models, this_c["model_type"])(pretrained=True)

dataset = SelfStorageImageDataset(image_dir, ll_formatted)

print(len(dataset))

results_dict = full_featurize(dataset, model_ft, this_c["batch_size"])
results_dict["latlon"] = ll_formatted
results_dict["ids_X"] = places_ids
with open(out_file, "wb") as f:
    dill.dump(results_dict, f, protocol=4)

2296
18
batch: 0
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 0 took 13.816879034042358
batch: 1
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 1 took 13.72326946258545
batch: 2
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 2 took 13.644095420837402
batch: 3
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 3 took 13.666235208511353
batch: 4
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 4 took 13.621318578720093
batch: 5
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 5 took 13.620375156402588
batch: 6
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 6 took 13.611730098724365
batch: 7
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 7 took 13.635680675506592
batch: 8
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 8 took 13.63889455795288
batch: 9
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 9 took 13.639105558395386
batch: 10
torch.Size([128, 3, 640, 640])
(128, 224, 224, 3)
batch: 10 took

In [206]:
# see that we can load the data correctly

with open(out_file, "rb") as f:
    data = np.load(f, allow_pickle=True)
print(len(data["X"]))
print(type(data["X"]))
print(len(data["latlon"]))
temp = b_io.BytesIO(data["X"])
features = np.load(temp, allow_pickle=True)
print(features.shape)
latlons_samp = data["latlon"]
ids_x = data["ids_X"]

X = pd.DataFrame(features, index=ids_x)
lls = pd.DataFrame(latlons_samp, index=ids_x, columns=["lat", "lon"])

lls
X

18808960
<class 'bytes'>
2296
(2296, 2048)


,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
ChIJkxtdA8mzQIYROlwVeGwwh6I,0.834773,0.643636,0.303390,0.507356,0.267298,0.759907,0.086311,0.650967,0.491964,0.715890,...,1.228650,0.310651,1.727601,0.567876,0.466545,0.562729,0.674225,0.171659,0.323485,0.145496
ChIJBXG-W2DBIogRs552_nWUOws,0.501546,0.277862,0.558959,1.154635,0.425419,0.804184,0.968238,0.716649,0.972749,0.648332,...,0.678585,1.269697,0.094451,0.104070,0.292546,0.602512,0.278408,0.693658,1.030975,1.022443
ChIJJ6ZCoiSCRoYRjFNyzETgjMc,1.774633,0.374899,0.109213,0.094782,0.071146,1.167407,0.880647,0.664006,0.360140,0.322937,...,0.079252,1.228090,0.045673,0.265189,0.584318,0.588619,0.400455,0.543755,0.282055,0.378244
ChIJGxPXPm9TrIkR8DTsN0sjT24,1.021847,0.135221,0.170440,0.172202,0.004961,0.100990,1.062565,0.354749,0.148183,0.123014,...,0.048577,1.344150,0.051268,0.105018,0.101661,0.252059,0.279709,0.207649,0.025371,0.012597
ChIJQe7mgp2OyIkRyP-yL8dVN00,0.843260,0.161551,0.220457,0.280847,0.200190,0.187166,0.285639,0.131765,0.641242,0.254351,...,0.398224,0.392477,0.407659,0.104021,0.065402,0.907138,0.417078,0.738831,0.197593,0.150824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJRR3QlPfX54kRk_-jdB-UnJk,0.973546,0.156330,0.428275,0.674488,0.160263,0.099020,0.263523,0.758507,1.149200,0.193240,...,0.180849,0.726241,0.383875,0.396036,0.453375,0.698190,1.289613,0.065364,0.466287,0.125149
ChIJ176SKpF744kRqd4b-EaZuW4,1.169964,0.789105,2.661240,0.342983,0.867167,1.228779,0.992280,0.967125,0.703880,0.358594,...,0.787595,0.151016,0.301524,0.518473,0.450908,1.006909,0.401902,0.196315,0.511094,0.097903
ChIJQa049JwR3okRPMVp5oeCFns,1.617521,0.062766,0.436373,0.329177,0.311482,0.330093,1.144737,0.022136,0.199334,0.695969,...,0.769487,0.579600,0.311243,0.317277,0.523552,0.060531,0.706488,0.238696,0.160066,0.862279
ChIJr_-EegO354kRitPgb5lyFmo,0.733202,0.956420,2.038173,0.327078,0.211475,0.414616,0.194798,0.410413,1.675975,0.769311,...,0.329809,0.494127,0.457495,0.149271,0.065779,0.363151,0.168935,0.062879,0.363713,0.407139
